In [ ]:
# Setup cell - Install compatible versions
!pip install -U "transformers>=4.35.0,<4.40.0" "accelerate>=0.20.0" "safetensors>=0.3.0" "huggingface_hub>=0.16.0" "bitsandbytes>=0.41.0,<0.44.0"

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Load requirements
!cd /kaggle/working/kaggleproject
!pip install --upgrade --target /kaggle/working/vendor -r requirementupdated_fixed.txt

In [ ]:
# Add vendor to sys.path
import sys
sys.path.append('/kaggle/working/vendor')

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# Load the model with compatible configuration
model_id = 'openai/gpt-oss-20b'

# Use compatible quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load model with error handling
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map='auto',
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    # Fallback to CPU loading if GPU fails
    print("Attempting CPU fallback...")
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )

In [ ]:
# Run the discovery scan
!python run_discovery.py --mode quick --print-summary